# Aircraft Design - SimPleAC and Comparison to Geometric Programming

Here, we implement an aircraft design problem called "SimPleAC" proposed by Prof. Warren Hoburg and refined in Berk Ozturk's Master's thesis.

This design problem was initially formulated as a demonstration of the geometric programming (GP) approach that GPKit uses. Here, we instead solve with the automatic differentiation (AD) optimization approach that AeroSandbox uses, and then we make comparisons between the two.

The problem itself is documented in the following locations:

* Primary reference is the associated model in [GPLibrary](https://github.com/convexengineering/gplibrary/tree/master/gpkitmodels/SP/SimPleAC)
* Berk's thesis is [available here](https://dspace.mit.edu/handle/1721.1/115595?show=full).

We begin our modeling by initializing an optimization environment:

In [1]:
import aerosandbox as asb
import aerosandbox.numpy as np

opti = asb.Opti()

We continue by defining all of the constants in SimPleAC. For convenience, we define these here all as simple scalars rather than parameters.

If we were interested in studying parametric sensitivities with respect to these constants, we could just as easily define these as parameters using `opti.parameter()` instead, and get the associated duals with `opti.dual(opti.parameter())`.

Note: we have converted all units into base SI units where applicable. (km -> m, etc.)

In [2]:
##### Constants

### Env. constants
g = 9.81  # gravitational acceleration, m/s^2
mu = 1.775e-5  # viscosity of air, kg/m/s
rho = 1.23  # density of air, kg/m^3
rho_f = 817  # density of fuel, kg/m^3

### Non-dimensional constants
C_Lmax = 1.6  # stall CL
e = 0.92  # Oswald's efficiency factor
k = 1.17  # form factor
N_ult = 3.3  # ultimate load factor
S_wetratio = 2.075  # wetted area ratio
tau = 0.12  # airfoil thickness to chord ratio
W_W_coeff1 = 2e-5  # wing weight coefficient 1
W_W_coeff2 = 60  # wing weight coefficient 2

### Dimensional constants
Range = 1000e3  # aircraft range, m
TSFC = 0.6 / 3600  # thrust specific fuel consumption, 1/sec
V_min = 25  # takeoff speed, m/s
W_0 = 6250  # aircraft weight excluding wing, N

We then implement the free variables. Here, we do this basically the same way as SimPleAC.

As a side note: upon inspection, it's *very clear* that a lot of the quantities listed as "free variables" in the SimPleAC source are extraneous and can be explicitly computed with minimal effort. Examples:
* Reynolds number `Re` - there is no reason to declare this as a design variable when it can instead be explicitly computed as a function of cruise speed and wing geometry.
* Lift to drag ratio `LoD` - this is defined as a variable and then constrained to be equal to `L / D`, and then subsequently never used anywhere else in the problem formulation.
* `p_labor`, the labor cost, which is declared and then never used in the formulation.

Because of this, we remove a lot of the variable declarations with trivial effort, and **we solve the same engineering problem with much simpler notation**.

We give initial guesses based on our intuition for order of magnitude. (Note for later readers: these initial guesses were made based on engineering intuition *before* looking at the optimal solution.)

We can even exploit log-convexity of the problem by flagging `log_transform=True` on variables that we intuitively expect to be nicely behaved in log-space. (However, unlike a pure geometric programming approach, we are not required to do this for every variable, and this in no way restricts us to GP-compatibility later on when we write constraints or objectives.)

In [3]:
### Free variables (same as SimPleAC, with extraneous variables removed)
AR = opti.variable(init_guess=10, log_transform=True)  # aspect ratio
S = opti.variable(init_guess=10, log_transform=True)  # total wing area, m^2
V = opti.variable(init_guess=100, log_transform=True)  # cruise speed, m/s
W = opti.variable(init_guess=10000, log_transform=True)  # total aircraft weight, N
C_L = opti.variable(init_guess=1, log_transform=True)  # lift coefficient
W_f = opti.variable(init_guess=3000, log_transform=True)  # fuel weight, N
V_f_fuse = opti.variable(
    init_guess=1, log_transform=True
)  # fuel volume in the fuselage, m^3

Our rules of thumb when deciding whether to log-transform:
* Is this variable pretty much exclusively used in power-law-type relations?
* Do we think a singularity would occur if this variable went to exactly zero, or negative?

So, we choose to log-transform:
* `V`, because we expect bad things to happen with our $C_f$ model if `Re` goes to zero.
* `C_L`, because if $C_L$ goes to zero, $L=W$ cannot occur regardless of airspeed (and if $C_L$ goes negative, the lift-to-airspeed proportionality goes negative)
* `S`, because if $S$ goes to zero, the same thing as above occurs.
* `AR`, because if $AR$ goes to zero, we have a span of zero which we expect to cause a singularity.

For the purposes of illustration, we log-transform the rest of the variables here too.

We implement the weight and lift models:

In [4]:
### Wing weight
W_w_surf = W_W_coeff2 * S
W_w_strc = (
    W_W_coeff1 / tau * N_ult * AR**1.5 * np.sqrt((W_0 + V_f_fuse * g * rho_f) * W * S)
)
W_w = W_w_surf + W_w_strc

### Entire weight
opti.subject_to(W >= W_0 + W_w + W_f)

### Lift equals weight constraint
opti.subject_to(
    [
        W_0 + W_w + 0.5 * W_f <= 0.5 * rho * S * C_L * V**2,
        W <= 0.5 * rho * S * C_Lmax * V_min**2,
    ]
)

### Flight duration
T_flight = Range / V

We implement the thrust and drag model:

In [5]:
### Drag
Re = (rho / mu) * V * (S / AR) ** 0.5
C_f = 0.074 / Re**0.2

CDA0 = V_f_fuse / 10

C_D_fuse = CDA0 / S
C_D_wpar = k * C_f * S_wetratio
C_D_ind = C_L**2 / (np.pi * AR * e)
C_D = C_D_fuse + C_D_wpar + C_D_ind
D = 0.5 * rho * S * C_D * V**2

opti.subject_to(
    [
        W_f >= TSFC * T_flight * D,
    ]
)

[MX(fabs(opti0_lam_g_4))]

We implement the fuel volume model:

In [6]:
V_f = W_f / g / rho_f
V_f_wing = 0.03 * S**1.5 / AR**0.5 * tau  # linear with b and tau, quadratic with chord

V_f_avail = V_f_wing + V_f_fuse

opti.subject_to(V_f_avail >= V_f)

MX(fabs(opti0_lam_g_5))

We implement an objective, and we solve:

In [7]:
opti.minimize(W_f)

sol = opti.solve(max_iter=100)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       24
Number of nonzeros in Lagrangian Hessian.............:       19

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        5

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.0000000e+03 3.85e+03 1.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

So, the solution was found in around 10-15 milliseconds.

Let's print the value of our variables:

In [8]:
for value in [
    "V",
    "W",
    "C_L",
    "W_f",
    "AR",
    "S",
    "V_f_fuse",
]:
    print(f"{value:10} = {sol(eval(value)):.6}")

V          = 57.106
W          = 8704.82
C_L        = 0.290128
W_f        = 937.756
AR         = 12.1049
S          = 14.1542
V_f_fuse   = 0.0619038


And we can print the value of some auxiliary parameters too:

In [9]:
for value in [
    "CDA0",
    "C_D",
    "C_L",
    "C_f",
    "D",
    "C_L/C_D",
    "Re",
    "T_flight",
    "V_f",
    "V_f_wing",
    "V_f_avail",
    "W_f",
    "W_w",
    "W_w_strc",
    "W_w_surf",
]:
    print(f"{value:10} = {sol(eval(value)):.6}")

CDA0       = 0.00619038
C_D        = 0.0113188
C_L        = 0.290128
C_f        = 0.00349109
D          = 321.309
C_L/C_D    = 25.6325
Re         = 4.27908e+06
T_flight   = 17511.3
V_f        = 0.117003
V_f_wing   = 0.0550997
V_f_avail  = 0.117003
W_f        = 937.756
W_w        = 1517.06
W_w_strc   = 667.811
W_w_surf   = 849.25


## GPKit Comparison

The optimal values obtained here exactly match the optimum values computed in the original source using GPKit; output below. Source code for the associated GPKit run is [here](./Ignore/SimPleAC - GPKit Implementation.py).

```
Starting a sequence of GP solves
 for 4 free variables
  in 1 locally-GP constraints
  and for 21 free variables
       in 22 posynomial inequalities.
Solving took 0.117 seconds and 2 GP solves.
Optimal Cost
------------
 937.8
Free Variables
--------------
       (CDA0) : 0.00619    [m**2] fuselage drag area
            A : 12.1              aspect ratio
          C_D : 0.01132           drag coefficient
          C_L : 0.2901            lift coefficient of wing
          C_f : 0.003491          skin friction coefficient
            D : 321.3      [N]    total drag force
          L/D : 25.63             lift-to-drag ratio
           Re : 4.279e+06         Reynold's number
            S : 14.15      [m**2] total wing area
   T_{flight} : 4.864      [hr]   flight time
            V : 57.11      [m/s]  cruising speed
          V_f : 0.117      [m**3] fuel volume
     V_f_fuse : 0.0619     [m**3] fuel volume in the fuselage
     V_f_wing : 0.0551     [m**3] fuel volume in the wing
V_{f_{avail}} : 0.117      [m**3] fuel volume available
            W : 8705       [N]    total aircraft weight
          W_f : 937.8      [N]    fuel weight
          W_w : 1517       [N]    wing weight
     W_w_strc : 667.8      [N]    wing structural weight
     W_w_surf : 849.2      [N]    wing skin weight
```

However, not only is the automatic differentiation (AD) approach more generalizable that the GP approach; in this demo the AD approach is roughly an order of magnitude faster on the same hardware.

Granted, this isn't a true head-to-head speed comparison, as we removed extraneous variables here. And, if we were to track parametric sensitivities as well, the speed advantage of AD would also shrink. However, the AD approach is clearly *at the very least* competitive speed-wise with GP while being vastly more generalizable to non-GP, nonconvex problems.

At the same time, we do need to concede a drawback of the AD approach over the GP approach:
* The AD approach is not guaranteed to converge for all initial guesses; indeed, if one guesses random values that aren't within 1 or 2 orders of magnitude of the optimal values, convergence tends to be difficult. However, on engineering design problems, we can usually easily make good enough guesses based on intuition. As an example, based on SimPleAC:
    * Recall that the optimal cruise airspeed is 57 m/s, and our naive initial guess for this value was 100 m/s.
    * If we change the initial guess to 1,000 m/s (Mach ~3), it still converges easily to the same optimum in just 31 iterations.
    * If we change the initial guess to 10,000 m/s (Mach ~30), it does not converge.

Another stated advantage of the GP approach is that global optimality is guaranteed; however, I argue that this is a misleading claim:
* If the true underlying problem is unimodal (i.e. there is only one local optimum, which is the global optimum), then any solution returned by either GPKit or a modern NLP solver (like IPOPT, which AeroSandbox uses), will be globally optimal.
    * If the initial guesses or scaling are poor, the NLP solver might fail to find a solution in a case where GPKit would not fail; however, this is a different problem than the NLP solver returning a solution that is locally-but-not-globally optimal.
* If the true underlying problem is multimodal, then:
    * In the GP approach: the problem is, by definition, not GP-compatible as-is. Therefore, we must force the problem to be GP-compatible by:
        * Surrogate modeling, in which case we are solving a different, unimodal approximation of the original problem. Hence, we're not solving the true engineering problem so global optimality guaranatees are meaningless.
            * Example: If a problem has a subsonic optimum and a supersonic optimum, and we force it to be GP-compatible by fitting a GP-compatible model to the subsonic branch, we can no longer make global optimality claims of the true underlying problem.
        * Signomial programming, in which case we give up global optimality guarantees anyway.
    * In the AD approach: we have no global optimality guarantees.

In other words: if you solve a multimodal problem by approximating it as a unimodal one, then of course you are going to get the global optimum of your approximation - this is true whether you use a GP solution or an NLP solution. But that doesn't mean you can claim global optimality on the original problem!

Hence, I'd argue that both the GP approach and the AD approach have identically strong claims on whether their solutions are "globally optimal", regardless of whether the underlying engineering problem is GP-compatible or not.